In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [70]:
def scrape_webull_headlines(ticker, date_tracker, start_phrase, end_phrase, scraped_headers, scraped_dates, file_name):

    # set up the webdriver
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)

    # navigate to the webpage
    driver.get("https://www.webull.com/newslist/nasdaq-" + ticker)

    continue_scrape = False
    # date_tracker = '03/27' #start date

    while True:
        
        try:
            more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
                                    (By.XPATH, "//div[@class='csr19'][text()='More']")))
            more_button.click()
            time.sleep(1)

            headers = driver.find_elements_by_class_name("csr14")
            dates = driver.find_elements_by_class_name("csr15")

            for i in range(len(headers)):
                header = headers[i]
                date = dates[i]

                # extract relevant items
                header = header.text
                date = date.text.split('·')[1].strip()
                date = date.split(' ')[0]

                # print(date)
                # print("date_tracker: ", date_tracker)

                if header == start_phrase and (len(date) == 10 and date[2] == '/' and date[5] == '/') and str(date) <= date_tracker and header not in scraped_headers:
                    continue_scrape = True
                    scraped_headers.append(header)
                    scraped_dates.append(date)
                    date_tracker = str(date)
                    print("Scraping starts!...", header, date)

                if header == end_phrase:
                    scraped_headers.append(header)
                    scraped_dates.append(date)
                    print("Scraping ends!...", header, date)
                    driver.quit()
                    break

                if continue_scrape and (len(date) == 10 and date[2] == '/' and date[5] == '/') and str(date) <= date_tracker and header not in scraped_headers:
                    scraped_headers.append(header)
                    scraped_dates.append(date)
                    date_tracker = str(date)
                    print("Scraping...", header, date)
        except:
            driver.quit()
            break
    
    # Combine the two lists into a DataFrame
    df = pd.DataFrame({'date': scraped_dates, 'news_headline': scraped_headers})

    # Save the DataFrame as a CSV file
    df.to_csv(file_name, index=False)

webull scrapes from:
- yahoo finance
- bloomber
- benziga
- reuters
- seeking alpha
- american city business journals
- zacks

In [71]:
# run the function

# 399 minutes
# last 2022 post (dec 30): Tracking Philippe Laffont's Coatue Management Portfolio - Q3 2022 Update
# first 2022 post (jan 1): 3 Growth Stocks Poised for a Bull Run in 2022

# ? mins
# last 2021 post (dec 31):  Zoom Is Betting Virtual Events Are Here to Stay
# first 2021 post (jan 4): Slinger Bag Has Created A Product That Allows Tennis Players To Practice Anytime, Anywhere

# last 2020 post (dec 31):  Tesla leads list of 2020's biggest winners on both S&P 500, Nasdaq 100
# first 2020 post (jan 3): Last year's Bay Area IPO performances show why it pays to be an insider


start_phrase = "Zoom Is Betting Virtual Events Are Here to Stay" 
end_phrase = "Slinger Bag Has Created A Product That Allows Tennis Players To Practice Anytime, Anywhere"

# start_phrase = "BRIEF-Zoom Announces Expansion Of Zoom IQ"
# end_phrase = "Better Buy: Zoom vs. Okta" 
scraped_headers = []
scraped_dates = []
file_name = 'full_webull_zoom_stock_news.csv'
ticker = 'zm'
date_tracker = '12/30/2022'

scrape_webull_headlines(ticker, date_tracker, start_phrase, end_phrase, scraped_headers, scraped_dates, file_name)

/var/folders/cf/28wm9krx5816hxprmj3lstxh0000gn/T/ipykernel_11467/3141270521.py:22: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  headers = driver.find_elements_by_class_name("csr14")
/var/folders/cf/28wm9krx5816hxprmj3lstxh0000gn/T/ipykernel_11467/3141270521.py:23: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  dates = driver.find_elements_by_class_name("csr15")


Scraping starts!... Tracking Philippe Laffont's Coatue Management Portfolio - Q3 2022 Update 12/30/2022
Scraping... Zoom shares climb 4% in effort to claim late-December gains 12/29/2022
Scraping... Insider Sell: Zoom Video Communications 12/29/2022
Scraping... What's Going On With Cathie Wood's ARK Innovation ETF (ARKK)? 12/28/2022
Scraping... Cathie Wood Channels Warren Buffett With Year End Investing Message 12/28/2022
Scraping... Cathie Wood Says Risk-Off Environment Pushed Investors Back Toward Benchmarks: 'While Innovation Solves Problems...' 12/28/2022
Scraping... Are Growth Investors Finally Cutting Ties With Cathie Wood? Ark Innovation Flows Vanish As Flagship Fund Hits 5-Year Low 12/20/2022
Scraping... Cathie Woods Ark Invest Buys $4.7M Of Tesla Stock And These 3 Tech Stocks Amid Friday's Market Weakness 12/17/2022
Scraping... What can we expect from meme stocks AMC, GameStop and Bed Bath & Beyond in 2023? 12/16/2022
Scraping... 3 Big Reasons to Buy Zoom Stock in 2023 12/16/2

In [55]:
def scrape_webull_headlines(ticker, start_date, end_date, scraped_headers, scraped_dates, file_name):

    # set up the webdriver
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)

    # navigate to the webpage
    driver.get("https://www.webull.com/newslist/nasdaq-" + ticker)

    continue_scrape = False
    date_tracker = start_date #start date

    while True:
        
        try:
            more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
                                    (By.XPATH, "//div[@class='csr19'][text()='More']")))
            more_button.click()
            time.sleep(1)

            headers = driver.find_elements_by_class_name("csr14")
            dates = driver.find_elements_by_class_name("csr15")

            for i in range(len(headers)):
                header = headers[i]
                date = dates[i]

                # extract relevant items
                header = header.text
                date = date.text.split('·')[1].strip()
                date = date.split(' ')[0]

                print(date)
                # print("date_tracker: ", date_tracker)

                if str(date) == start_date:
                    print("found 1: ", start_date)
                if str(date) == start_date and continue_scrape == False:
                    print("found 2: ", start_date)
                if str(date) == start_date and continue_scrape == False and str(date) <= date_tracker:
                    print("found 3: ", start_date)
                if str(date) == start_date and continue_scrape == False and str(date) <= date_tracker and header not in scraped_headers:
                    continue_scrape = True
                    scraped_headers.append(header)
                    scraped_dates.append(date)
                    date_tracker = str(date)
                    print("Scraping starts!...", header, date)

                if str(date) == end_date or str(date) == '01/02/2020' or str(date) == '12/30/2019' or str(date) == '12/29/2019':
                    print("Scraping ends!...", header, date)
                    driver.quit()
                    break

                if continue_scrape and str(date) <= date_tracker and header not in scraped_headers:
                    scraped_headers.append(header)
                    scraped_dates.append(date)
                    date_tracker = str(date)
                    print("Scraping...", header, date)
        except:
            driver.quit()
            break
    
    # Combine the two lists into a DataFrame
    df = pd.DataFrame({'date': scraped_dates, 'news_headline': scraped_headers})

    # Save the DataFrame as a CSV file
    df.to_csv(file_name, index=False)

In [56]:
# run the function
# start_phrase = "Rosenblatt Initiates Coverage On Zoom Video Comms with Neutral Rating, Announces Price Target of $75" # last article of dec 31 2022
# end_phrase = "How Zoom's Mission Could Help It Lower Its Biggest Expense" 


start_date = '12/30/2022' # last article of dec 31, 2022
end_date = '01/01/2020' # last article of jan 1, 2020
scraped_headers = []
scraped_dates = []
file_name = 'full_webull_zoom_stock_news.csv'
ticker = 'zm'

scrape_webull_headlines(ticker, start_date, end_date, scraped_headers, scraped_dates, file_name)

/var/folders/cf/28wm9krx5816hxprmj3lstxh0000gn/T/ipykernel_11467/1758405074.py:22: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  headers = driver.find_elements_by_class_name("csr14")
/var/folders/cf/28wm9krx5816hxprmj3lstxh0000gn/T/ipykernel_11467/1758405074.py:23: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  dates = driver.find_elements_by_class_name("csr15")


1h
12h
15h
17h
17h
17h
20h
1d
1d
1d
1d
1d
1d
4d
4d
4d
4d
4d
4d
4d
4d
5d
04/10
04/07
04/07
04/06
04/05
04/05
04/02
03/31
03/30
03/30
03/30
03/29
03/29
03/29
03/28
03/28
03/28
1h
12h
15h
17h
17h
17h
20h
1d
1d
1d
1d
1d
1d
4d
4d
4d
4d
4d
4d
4d
4d
5d
04/10
04/07
04/07
04/06
04/05
04/05
04/02
03/31
03/30
03/30
03/30
03/29
03/29
03/29
03/28
03/28
03/28
03/27
03/27
03/27
03/25
03/24
03/24
03/24
03/24
03/24
03/24
03/23
03/23
03/23
03/21
03/21
03/17
03/17
03/17
03/17
03/17
1h
12h
15h
17h
17h
17h
20h
1d
1d
1d
1d
1d
1d
4d
4d
4d
4d
4d
4d
4d
4d
5d
04/10
04/07
04/07
04/06
04/05
04/05
04/02
03/31
03/30
03/30
03/30
03/29
03/29
03/29
03/28
03/28
03/28
03/27
03/27
03/27
03/25
03/24
03/24
03/24
03/24
03/24
03/24
03/23
03/23
03/23
03/21
03/21
03/17
03/17
03/17
03/17
03/17
03/16
03/16
03/16
03/15
03/15
03/15
03/14
03/14
03/14
03/14
03/13
03/11
03/11
03/10
03/10
03/09
03/07
03/07
03/07
03/07
1h
12h
15h
17h
17h
17h
20h
1d
1d
1d
1d
1d
1d
4d
4d
4d
4d
4d
4d
4d
4d
5d
04/10
04/07
04/07
04/06
04/05
04/05
04/02
03/3

In [26]:
# def scrape_webull_headlines(ticker, start_date, end_date, scraped_headers, scraped_dates, file_name):

#     # set up the webdriver
#     options = webdriver.ChromeOptions()
#     options.add_argument("--start-maximized")
#     driver = webdriver.Chrome(options=options)

#     # navigate to the webpage
#     driver.get("https://www.webull.com/newslist/nasdaq-" + ticker)

#     continue_scrape = False

#     while True:
        
#         try:
#             more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
#                                     (By.XPATH, "//div[@class='csr19'][text()='More']")))
#             more_button.click()
#             time.sleep(1)

#             headers = driver.find_elements_by_class_name("csr14")
#             dates = driver.find_elements_by_class_name("csr15")

#             for i in range(len(headers)):
#                 header = headers[i]
#                 date = dates[i]

#                 # extract relevant items
#                 header = header.text
#                 date = date.text.split('·')[1].strip()
#                 date = date.split(' ')[0]
#                 print("extracted date: ", date)

#                 if date == start_date and continue_scrape == False:
#                     continue_scrape = True
#                     scraped_headers.append(header)
#                     scraped_dates.append(date)
#                     print("Scraping starts!...", header, date)

#                 if date == end_date:
#                     print("Scraping ends!...", header, date)
#                     driver.quit()
#                     # print(scraped_headers[1:])
#                     # print(scraped_dates[1:])
#                     break

#                 if continue_scrape:
#                     scraped_headers.append(header)
#                     scraped_dates.append(date)
#                     print("Scraping...", header, date)
#         except:
#             driver.quit()
#             break
    
#     # Combine the two lists into a DataFrame
#     df = pd.DataFrame({'date': scraped_dates[1:], 'news_headline': scraped_headers[1:]})

#     # Save the DataFrame as a CSV file
#     df.to_csv(file_name, index=False)

In [27]:
# # run the function
# # start_phrase = "Rosenblatt Initiates Coverage On Zoom Video Comms with Neutral Rating, Announces Price Target of $75" # last article of dec 31 2022
# # end_phrase = "How Zoom's Mission Could Help It Lower Its Biggest Expense" 


# start_date = '12/31/2022' # last article of dec 31, 2022
# end_date = '12/26/2022' # last article of dec 31, 2019
# scraped_headers = []
# scraped_dates = []
# file_name = 'full_webull_zoom_stock_news.csv'
# ticker = 'zm'

# scrape_webull_headlines(ticker, start_date, end_date, scraped_headers, scraped_dates, file_name)

/var/folders/cf/28wm9krx5816hxprmj3lstxh0000gn/T/ipykernel_11467/4290478050.py:21: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  headers = driver.find_elements_by_class_name("csr14")
/var/folders/cf/28wm9krx5816hxprmj3lstxh0000gn/T/ipykernel_11467/4290478050.py:22: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  dates = driver.find_elements_by_class_name("csr15")


extracted date:  20m
extracted date:  11h
extracted date:  14h
extracted date:  16h
extracted date:  16h
extracted date:  16h
extracted date:  19h
extracted date:  1d
extracted date:  1d
extracted date:  1d
extracted date:  1d
extracted date:  1d
extracted date:  1d
extracted date:  4d
extracted date:  4d
extracted date:  4d
extracted date:  4d
extracted date:  4d
extracted date:  4d
extracted date:  4d
extracted date:  4d
extracted date:  5d
extracted date:  04/10
extracted date:  04/07
extracted date:  04/07
extracted date:  04/06
extracted date:  04/05
extracted date:  04/05
extracted date:  04/02
extracted date:  03/31
extracted date:  03/30
extracted date:  03/30
extracted date:  03/30
extracted date:  03/29
extracted date:  03/29
extracted date:  03/29
extracted date:  03/28
extracted date:  03/28
extracted date:  03/28
extracted date:  20m
extracted date:  11h
extracted date:  14h
extracted date:  16h
extracted date:  16h
extracted date:  16h
extracted date:  19h
extracted date: